# Web data from __[telekom.de](https://www.telekom.de/hilfe)__


### Description
This function processes `.txt` files in a specified directory to extract relevant information and write it to different output files based on their content. It categorizes the files into three categories:
1. **Files with valid questions and answers**: Written to `web_data.txt`.
2. **Files that could not be processed**: Written to `filtered.txt`.
3. **Files with questions containing `?` but not ending with `?`**: Written to `inspecting.txt`.

### Inputs
- **`directory_path`**: Path to the directory containing `.txt` files.
- **`web_data_file`**: Path to the output file for valid questions and answers.
- **`filtered_file`**: Path to the output file for files that couldn't be processed.
- **`inspecting_file`**: Path to the output file for questions containing `?` but not ending with `?`.

### Outputs
- **`web_data.txt`**: Contains the valid question and answer data.
- **`filtered.txt`**: Contains filenames of files that couldn't be processed.
- **`inspecting.txt`**: Contains questions with `?` but not ending with `?` along with their details.


In [ ]:
import re
import os

# Define fixed paths for the input directory and output files
directory_path = "/Users/taha/Desktop/scrapeV2/output_folder"
web_data_directory = "web_data"
web_data_file = os.path.join(web_data_directory, "web_data.txt")
filtered_file = os.path.join(web_data_directory, "filtered.txt")
inspecting_file = os.path.join(web_data_directory, "inspecting.txt")

def extract_and_write_data(directory_path, web_data_file, filtered_file, inspecting_file):
    """
    Extracts and categorizes data from .txt files in the specified directory.
    
    Parameters:
    - directory_path (str): Path to the directory containing .txt files.
    - web_data_file (str): Path to the output file for valid questions and answers.
    - filtered_file (str): Path to the output file for files that couldn't be processed.
    - inspecting_file (str): Path to the output file for questions containing `?` but not ending with `?`.
    
    Outputs:
    - Writes valid questions and answers to web_data_file.
    - Writes filenames of unprocessed files to filtered_file.
    - Writes questions with `?` but not ending with `?` to inspecting_file.
    """

    # Regex pattern to find the section starting with "...Telekom" and ending with two spaces
    pattern = r"\.\.\.Telekom.*?\s{2}"
    
    # Initialize counters and lists to keep track of files and their processing
    total_files = 0
    processed_files = 0
    processed_files_data = []
    unprocessed_files = []
    inspecting_files = []

    def process_file(filename):
        """
        Processes each .txt file to extract and categorize content based on patterns.
        
        Parameters:
        - filename (str): The name of the file to be processed.
        """
        nonlocal processed_files
        file_path = os.path.join(directory_path, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as source:
                content = source.read()
                match = re.search(pattern, content, re.DOTALL)
                if match:
                    processed_files += 1
                    matched_text = match.group(0)
                    start_index = match.end()

                    # Clean and format the matched text
                    cleaned_text = re.sub(r'\.\.\.+', '\n', matched_text.strip(".").strip())
                    
                    # Extract the text after the matched pattern
                    post_pattern_text = content[start_index:]
                    
                    # Find paragraphs separated by multiple newlines
                    paragraph_pattern = r'([^\n]+(?:\n[^\n]+)*)(?:\n{2,})'
                    paragraphs = re.findall(paragraph_pattern, post_pattern_text)
                    
                    if len(paragraphs) >= 2:
                        question = paragraphs[0].strip()
                        answer = paragraphs[1].strip()
                        
                        if question.endswith('?'):
                            processed_files_data.append((filename, cleaned_text, question, answer))
                        else:
                            if '?' in question:
                                inspecting_files.append(f"File: {filename}\nNavigation:\n{cleaned_text}\n\nQuestion: {question}\n\nAnswer: {answer}\n")
                    else:
                        unprocessed_files.append(filename)
                else:
                    unprocessed_files.append(filename)
        except Exception as e:
            unprocessed_files.append(filename)

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Directory {directory_path} does not exist.")
        return

    # Create web_data directory if it does not exist
    os.makedirs(web_data_directory, exist_ok=True)

    # Process each .txt file in the directory
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith(".txt"):
            total_files += 1
            process_file(filename)

    # Write valid question and answer data to web_data_file
    with open(web_data_file, "w", encoding="utf-8") as web_data_txt:
        for filename, navigation, question, answer in sorted(processed_files_data, key=lambda x: x[0]):
            web_data_txt.write(f"File: {filename}\n")
            web_data_txt.write(f"Navigation:\n{navigation}\n\n")
            web_data_txt.write(f"Question: {question}\n\n")
            web_data_txt.write(f"Answer: {answer}\n")
            web_data_txt.write("\n" + "="*40 + "\n")

    # Write filenames of unprocessed files to filtered_file
    with open(filtered_file, "w", encoding="utf-8") as filtered_txt:
        for filename in sorted(unprocessed_files):
            filtered_txt.write(f"{filename}\n")

    # Write questions with `?` but not ending with `?` to inspecting_file
    with open(inspecting_file, "w", encoding="utf-8") as inspecting_txt:
        inspecting_txt.writelines(inspecting_files)

    # Calculate numbers for print statements
    num_unprocessed_files = len(unprocessed_files)
    num_inspecting_files = len(inspecting_files)
    num_processed_files = len(processed_files_data)
    missing_files = total_files - (num_processed_files + num_unprocessed_files + num_inspecting_files)

    # Print summary of the processing
    print("Process completed.")
    print(f"Total number of .txt files in the folder: {total_files}")
    print(f"Number of .txt files processed and written to {web_data_file}: {num_processed_files}")
    print(f"Unprocessed files have been written to {filtered_file}. Number of unprocessed files: {num_unprocessed_files}")
    print(f"Files with questions containing '?' but not ending with '?' have been written to {inspecting_file}. Number of inspecting files: {num_inspecting_files}")
    print(f"Number of missing or unaccounted files: {missing_files}")

# Call the function with the specified parameters
extract_and_write_data(directory_path, web_data_file, filtered_file, inspecting_file)


In [8]:
import re
import os
import logging

# Setup logging
logging.basicConfig(filename='processing.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define fixed paths for the input directory and output file
directory_path = "/Users/taha/Desktop/scrapeV2/output_folder"
web_data_directory = "web_data"
navigation_file = os.path.join(web_data_directory, "navigation.txt")

def format_navigation_text(text):
    """
    Formats the navigation text by replacing newlines and multiple spaces with ' > '.
    
    Parameters:
    - text (str): The text to be formatted.
    
    Returns:
    - str: The formatted text with ' > ' as separators.
    """
    # Replace newlines and multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Replace single spaces with ' > ' for navigation format
    formatted_text = re.sub(r'\s* \s*', ' > ', text)
    
    return formatted_text

def extract_navigation_data(directory_path, navigation_file):
    """
    Extracts navigation data from .txt files in the specified directory and writes it to an output file.
    
    Parameters:
    - directory_path (str): Path to the directory containing .txt files.
    - navigation_file (str): Path to the output file for navigation data.
    
    Outputs:
    - Writes navigation data to navigation_file.
    """

    # Regex pattern to find the section starting with "...Telekom" and ending with two spaces
    pattern = r"\.\.\.Telekom.*?\s{2}"
    
    # Initialize lists to keep track of processed and unprocessed files
    processed_files_data = []
    unprocessed_files = []

    def process_file(filename):
        """
        Processes each .txt file to extract navigation content based on patterns.
        
        Parameters:
        - filename (str): The name of the file to be processed.
        """
        file_path = os.path.join(directory_path, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as source:
                content = source.read()
                match = re.search(pattern, content, re.DOTALL)
                if match:
                    matched_text = match.group(0)
                    start_index = match.end()

                    # Clean and format the matched text
                    cleaned_text = re.sub(r'\.\.\.+', '\n', matched_text.strip(".").strip())
                    
                    # Extract the text after the matched pattern
                    post_pattern_text = content[start_index:]
                    
                    # Find paragraphs separated by multiple newlines
                    paragraph_pattern = r'([^\n]+(?:\n[^\n]+)*)(?:\n{2,})'
                    paragraphs = re.findall(paragraph_pattern, post_pattern_text)
                    
                    if len(paragraphs) >= 1:
                        # Format navigation text
                        navigation = format_navigation_text(cleaned_text)
                        processed_files_data.append((filename, navigation))
                    else:
                        unprocessed_files.append(filename)
                else:
                    unprocessed_files.append(filename)
        except Exception as e:
            logging.error(f"Error processing file {filename}: {e}")
            unprocessed_files.append(filename)

    # Ensure the directory exists
    if not os.path.exists(directory_path):
        logging.error(f"Directory {directory_path} does not exist.")
        return

    # Create web_data directory if it does not exist
    os.makedirs(web_data_directory, exist_ok=True)

    # Process each .txt file in the directory
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith(".txt"):
            process_file(filename)

    # Write navigation data to the navigation_file
    with open(navigation_file, "w", encoding="utf-8") as nav_file:
        for filename, navigation in sorted(processed_files_data, key=lambda x: x[0]):
            # Prepend "https_" to the filename
            nav_file.write(f"https_{filename}\n")
            nav_file.write(f"{navigation}\n")
            nav_file.write("\n" + "="*40 + "\n")

    # Print summary of the processing
    total_files = len(processed_files_data) + len(unprocessed_files)
    num_processed_files = len(processed_files_data)
    num_unprocessed_files = len(unprocessed_files)
    missing_files = total_files - (num_processed_files + num_unprocessed_files)

    print("Process completed.")
    print(f"Total number of .txt files in the folder: {total_files}")
    print(f"Number of .txt files processed and written to {navigation_file}: {num_processed_files}")
    print(f"Unprocessed files: {num_unprocessed_files}")
    print(f"Number of missing or unaccounted files: {missing_files}")

    # Log the results
    logging.info(f"Total files: {total_files}")
    logging.info(f"Processed files: {num_processed_files}")
    logging.info(f"Unprocessed files: {num_unprocessed_files}")
    logging.info(f"Missing files: {missing_files}")

# Call the function with the specified parameters
extract_navigation_data(directory_path, navigation_file)

Process completed.
Total number of .txt files in the folder: 2595
Number of .txt files processed and written to web_data/navigation.txt: 1676
Unprocessed files: 919
Number of missing or unaccounted files: 0
